In [9]:
!pip install -U sentence-transformers

  Using cached sentence_transformers-2.2.0-py3-none-any.whl
  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
  Using cached sentencepiece-0.1.96-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached huggingface_hub-0.5.1-py3-none-any.whl (77 kB)
  Using cached filelock-3.6.0-py3-none-any.whl (10.0 kB)
  Using cached tokenizers-0.12.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Using cached regex-2022.4.24-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (764 kB)
  Using cached sacremoses-0.0.49-py3-none-any.whl (895 kB)


In [2]:
!ls

nlp-pattern  nlp_pattern.ipynb


In [3]:
import os
os.chdir("nlp-pattern")

In [4]:
!git checkout dev

Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [ ]:
from clustering.fast_cluster import fast_cluster

fast_cluster("nuforc.txt")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Encode the corpus. This might take a while


Batches:   0%|          | 0/1389 [00:00<?, ?it/s]

[2022-04-25 22:04:11.462 pytorch-1-10-cpu-py38-ml-t3-medium-9c8fa3a7688fa232cbbe37185a11:159 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-04-25 22:04:11.637 pytorch-1-10-cpu-py38-ml-t3-medium-9c8fa3a7688fa232cbbe37185a11:159 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


/opt/conda/lib/python3.8/site-packages/smdebug-1.0.13b20220304-py3.8.egg/smdebug/profiler/system_metrics_reader.py:63: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.8/site-packages/smdebug-1.0.13b20220304-py3.8.egg/smdebug/profiler/system_metrics_reader.py:63: SyntaxWarning: "is not" with a literal. Did you mean "!="?


Start clustering


In [10]:
import time

from sentence_transformers import SentenceTransformer, util

from helpers import txt_to_list

In [7]:
import pickle

# Model for computing sentence embeddings.
model = SentenceTransformer('all-MiniLM-L6-v2')

corpus_sentences = txt_to_list("nuforc.txt")
print("Encode the corpus. This might take a while")
corpus_embeddings = model.encode(corpus_sentences, batch_size=64, show_progress_bar=True, convert_to_tensor=True)

with open('embeddings.pickle', 'ab') as token_file:        
    pickle.dump(corpus_embeddings, token_file)

Encode the corpus. This might take a while


Batches:   0%|          | 0/1389 [00:00<?, ?it/s]

[2022-04-25 23:25:48.395 pytorch-1-10-cpu-py38-ml-t3-medium-9c8fa3a7688fa232cbbe37185a11:556 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-04-25 23:25:48.598 pytorch-1-10-cpu-py38-ml-t3-medium-9c8fa3a7688fa232cbbe37185a11:556 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


/opt/conda/lib/python3.8/site-packages/smdebug-1.0.13b20220304-py3.8.egg/smdebug/profiler/system_metrics_reader.py:63: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/opt/conda/lib/python3.8/site-packages/smdebug-1.0.13b20220304-py3.8.egg/smdebug/profiler/system_metrics_reader.py:63: SyntaxWarning: "is not" with a literal. Did you mean "!="?


In [5]:
import pickle

with open("embeddings.pickle", 'rb') as token_file:
    corpus_embeddings = pickle.load(token_file)

In [ ]:
start_time = time.time()

#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 25 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(corpus_embeddings, min_community_size=25, threshold=0.75)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", corpus_sentences[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", corpus_sentences[sentence_id])

In [9]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 KB 1.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 KB 1.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.1/245.1 KB 4.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 KB 1.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 39.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 KB 2.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.6/561.6 KB 8.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 KB 1.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 KB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip list

Package                    Version
-------------------------- ---------------
argon2-cffi                21.3.0
argon2-cffi-bindings       21.2.0
asttokens                  2.0.5
attrs                      20.3.0
awscli                     1.22.68
awsio                      0.0.1
backcall                   0.2.0
bcrypt                     3.2.0
beautifulsoup4             4.11.1
black                      22.1.0
bleach                     5.0.0
bokeh                      2.4.2
boto3                      1.21.13
botocore                   1.24.13
Bottleneck                 1.3.2
brotlipy                   0.7.0
certifi                    2021.10.8
cffi                       1.14.6
charset-normalizer         2.0.4
click                      8.0.4
cloudpickle                2.0.0
colorama                   0.4.3
conda                      4.11.0
conda-package-handling     1.7.3
cryptography               36.0.1
cycler                     0.11.0
Cython                     0.29.21
decorator 